In [1]:
import tensorflow as tf

import pandas as pd
import codecs
import numpy as np
import math
import csv
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import recall_score, precision_score # New
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn import preprocessing
import matplotlib.pyplot as plt
import tensorflow.keras.preprocessing
#from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Input, Flatten, Add, Multiply, Subtract
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Activation, BatchNormalization
# regularizers
from tensorflow.keras.layers import Conv1D, Conv2D
from tensorflow.keras.layers import MaxPooling1D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import LSTM, Bidirectional
from tensorflow.keras.utils import plot_model
from tensorflow.keras.layers import Dropout, SpatialDropout1D
from scipy import signal
import openpyxl
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

from matplotlib import pylab as plt
import matplotlib.dates as mdates
import seaborn as sns

from keijzer import *
from tensorflow import keras
from keras.models import Model
from keras.layers import *
from tensorflow.keras.optimizers import Adam,SGD
from keras.layers import Dense, Input, Flatten, Add, concatenate, Dropout, Activation, Multiply, Embedding, Layer, Reshape
from keras.layers import LSTM, Bidirectional, GlobalMaxPooling1D, GlobalAveragePooling1D  
from keras.layers import Conv1D, AveragePooling1D, MaxPooling1D
#from keras.ops import convert_to_tensor, convert_to_numpy
#from keras.utils import plot_model
from keras import activations
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from keras.callbacks import ModelCheckpoint, CSVLogger, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
import math

def get_flops(model):
    run_meta = tf.RunMetadata()
    opts = tf.profiler.ProfileOptionBuilder.float_operation()

    # We use the Keras session graph in the call to the profiler.
    flops = tf.profiler.profile(graph=K.get_session().graph,
                                run_meta=run_meta, cmd='op', options=opts)

    return flops.total_float_ops  # Prints the "flops" of the model.

class LossHistory(tensorflow.keras.callbacks.Callback):  # history = LossHistory()
    def init(self):  # history.init()
        self.losses = []
        # self.accs = []
        self.val_losses = []
        # self.val_accs = []
        self.rmses = []
        self.mses = []
        self.maes = []
        self.mapes = []
        self.val_rmses = []
        self.val_mses = []
        self.val_maes = []
        self.val_mapes = []

    def on_epoch_end(self, batch, logs={}):
        self.losses.append(logs.get('loss'))
        # self.accs.append(logs.get('acc'))
        self.val_losses.append(logs.get('val_loss'))
        # self.val_accs.append(logs.get('val_accuracy
        self.rmses.append(logs.get('root_mean_squared_error'))
        self.mses.append(logs.get('mean_squared_error'))
        self.maes.append(logs.get('mean_absolute_error'))
        self.mapes.append(logs.get('mean_absolute_percentage_error'))
        self.val_rmses.append(logs.get('val_root_mean_squared_error'))
        self.val_mses.append(logs.get('val_mean_squared_error'))
        self.val_maes.append(logs.get('val_mean_absolute_error'))
        self.val_mapes.append(logs.get('val_mean_absolute_percentage_error'))

def mean_absolute_percentage_error(y_true, y_pred):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

def root_mean_squared_error(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 

In [2]:
import os

os.environ["CUDA_VISIBLE_DEVICES"] = '0' # 여러개 사용시 '0,1,2' 식으로 하나의 문자열에 입력
gpus = tf.config.experimental.list_physical_devices('GPU') # 호스트 러나임에 표시되는 GPU 장치 목록 반환

if gpus: # 반환된 GPU 장치 목록이 있다면
    try: # 해당 장치에 대한 메모리 증가 활성화 여부 설정
        tf.config.experimental.set_memory_growth(gpus[0], True)
    except RuntimeError as e: # try문 실패시에 에러문구 출력
        print(e)

print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [3]:
data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2020.csv'
data_source1 = pd.read_csv(data_path)
dat_source1 = pd.DataFrame(data_source1)
#dat_source1 = dat_source1.iloc[:-3,:]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2021.csv'
data_source2 = pd.read_csv(data_path)
dat_source2 = pd.DataFrame(data_source2)
dat_source2 = dat_source2.iloc[:8761,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2022.csv'
data_source3 = pd.read_csv(data_path)
dat_source3 = pd.DataFrame(data_source3)
dat_source3 = dat_source3.iloc[:8760,:9]

data_path = 'C:/Users/smartgrid_AI/Desktop/windpower_ramprate/ercot_2023.csv'
data_source4 = pd.read_csv(data_path)
dat_source4 = pd.DataFrame(data_source4)
dat_source4 = dat_source4.iloc[:8760,:9]

dat_arr1 = np.array(dat_source1)
dat_arr2 = np.array(dat_source2)
dat_arr3 = np.array(dat_source3)
dat_arr4 = np.array(dat_source4)

dat_arr = np.concatenate([dat_arr1, dat_arr2, dat_arr3, dat_arr4],axis=0)
dat_arr.shape # 8784, 8761, 8760, 8760

dat_source = pd.DataFrame(dat_arr, columns=['Time-Date stamp','Date','ERCOT Load','Total Wind Output','Total Wind Installed','Wind Output, % of Load','Wind Output, % of Installed','1-hr MW change','1-hr % change'])
dat_source

,Time-Date stamp,Date,ERCOT Load,Total Wind Output,Total Wind Installed,"Wind Output, % of Load","Wind Output, % of Installed",1-hr MW change,1-hr % change
0,01-01-2020 00:00:00,Jan-01,36823,12335,27040,33.5,45.617,NaN,NaN
1,01-01-2020 01:00:00,Jan-01,36511,13300,27040,36.4,49.187,965.4,7.826
2,01-01-2020 02:00:00,Jan-01,36022,13586,27040,37.7,50.245,286.2,2.152
3,01-01-2020 03:00:00,Jan-01,35790,14107,27040,39.4,52.171,520.9,3.834
4,01-01-2020 04:00:00,Jan-01,35642,14887,27040,41.8,55.055,779.6,5.527
...,...,...,...,...,...,...,...,...,...
35060,12-31-2023 20:00:00,Dec-31,44887,17161,38631,38.23,44.42,2858,19.99
35061,12-31-2023 21:00:00,Dec-31,44013,18301,38631,41.58,47.37,1139,6.64
35062,12-31-2023 22:00:00,Dec-31,43280,18607,38631,42.99,48.17,307,1.67
35063,12-31-2023 23:00:00,Dec-31,42319,17866,38631,42.22,46.25,-741,-3.98


In [4]:
dat = pd.DataFrame()
dat['MW % change'] = np.array(dat_source.iloc[1:,3], dtype=float)-np.array(dat_source.iloc[0:-1,3], dtype=float)
dat['MW % change'] = np.divide(np.array(dat['MW % change']),np.array(dat_source.iloc[1:,4]))*100
dat['Total Wind Output'] = np.divide(np.array(dat_source.iloc[1:,3]), np.array(dat_source.iloc[1:,4]))

dat

,MW % change,Total Wind Output
0,3.56879,0.491864
1,1.05769,0.502441
2,1.92678,0.521709
3,2.88462,0.550555
4,-0.580621,0.544749
...,...,...
35059,7.3982,0.444229
35060,2.951,0.473739
35061,0.79211,0.48166
35062,-1.91815,0.462478


## Figures

In [5]:
ramp_ratio = 20
load_ratio = 10
year_arr = np.concatenate([np.ones(8784)*2020, np.ones(8760)*2021, np.ones(8760)*2022, np.ones(8760)*2023])
year_df = pd.DataFrame(year_arr, columns=['Year'], dtype=int)
wind_df = pd.concat([year_df, dat], axis=1)
wind_df

,Year,MW % change,Total Wind Output
0,2020,3.56879,0.491864
1,2020,1.05769,0.502441
2,2020,1.92678,0.521709
3,2020,2.88462,0.550555
4,2020,-0.580621,0.544749
...,...,...,...
35059,2023,7.3982,0.444229
35060,2023,2.951,0.473739
35061,2023,0.79211,0.48166
35062,2023,-1.91815,0.462478


In [6]:
#from tensorflow.keras.layers import preprocessing
#from tensorflow.keras import preprocessing
import sklearn

wind_df = wind_df.reset_index(drop=True)
#std_scaler = sklearn.preprocessing.StandardScaler()
#std_scaler_load = sklearn.preprocessing.StandardScaler()
#std_scaler_ramp = sklearn.preprocessing.StandardScaler()

std_scaler = sklearn.preprocessing.MinMaxScaler()
std_scaler_load = sklearn.preprocessing.MinMaxScaler()
std_scaler_ramp = sklearn.preprocessing.MinMaxScaler()

wind_df[['Total Wind Output']] = std_scaler.fit_transform(wind_df[['Total Wind Output']])
wind_df[['MW % change']] = std_scaler_ramp.fit_transform(wind_df[['MW % change']])
#wind_df[['Wind Change, % of Load', 'Wind Change']] = std_scaler_ramp.fit_transform(wind_df[['Wind Change, % of Load', 'Wind Change']])
wind_df = wind_df.iloc[:-1,:]
wind_df

,Year,MW % change,Total Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


In [7]:
ramp_df = pd.DataFrame(wind_df)
ramp_df.columns=['Year', 'Wind Change', 'Wind Output']
ramp_df

,Year,Wind Change,Wind Output
0,2020,0.620197,0.644724
1,2020,0.544580,0.658617
2,2020,0.570751,0.683924
3,2020,0.599594,0.721813
4,2020,0.495246,0.714187
...,...,...,...
35058,2023,0.774643,0.484984
35059,2023,0.735512,0.582157
35060,2023,0.601593,0.620917
35061,2023,0.536583,0.631322


## Common Definitions

In [8]:
def create_dataset(dataset, timesteps, output_timesteps, leadtime):
    dataX, dataY = [], []
    for i in range(len(dataset) - timesteps -output_timesteps - leadtime - 1):
        a = dataset[i:(i + timesteps), :]
        dataX.append(a)
        dataY.append(dataset[(i + timesteps + leadtime):(i+timesteps+output_timesteps+leadtime), :])
    return np.array(dataX), np.array(dataY)

def svdadd(X, start, end, b_size):
    dec_x = X[start:end, :, :]
    U, S, Vh = svd(dec_x.reshape(b_size, -1), full_matrices=True)

    high_sig = np.matmul(np.matmul(U[:,:high_ind], np.diag(S[0:high_ind])), Vh[:high_ind,:])
    low_sig = X[:dec_num, :, :].reshape(b_size,-1)-high_sig

    rec_x = np.zeros((b_size, b_size, high_ind))
    
    #print(sum(S[0:high_ind])/sum(S))
    
    for i in range(high_ind):
        rec_x[:,:,i] = np.matmul((U[:,i]*S[i]).reshape(-1,1), Vh[i,:].reshape(1,-1))    
    return rec_x, low_sig

In [9]:
from sklearn.model_selection import train_test_split
from numpy.linalg import svd
import gc

def MAEMS(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true))*100

def MAEMD_1h(y_true, y_pred):
    return K.mean((K.abs(y_pred - y_true))*K.square(y_true))*100

def mse_mae(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))+K.mean(K.abs(y_pred - y_true))

from keras.layers.advanced_activations import PReLU

## Wind Generation Forecasting

In [10]:
norm_df = np.array(wind_df.iloc[:,2]).reshape(-1,1)
timesteps = 24
output_timesteps = 1
leadtime = 1
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [11]:
dec_num = timesteps
high_ind = 12

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

## Load and Test Pre-trained Basic Model

In [12]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)
trvaY = trvaY.reshape(-1,output_timesteps)
teY = teY.reshape(-1,output_timesteps)
trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31532, 24, 14), (31532, 1), (3504, 24, 14), (3504, 1))

In [13]:
with tf.device('/gpu:0'):
    b_size = output_timesteps*100
    wind_model = keras.models.load_model('Basic Model Final_fh1.h5', custom_objects={'MAEMS': MAEMS})
    winder_model = keras.models.load_model('Error Learning Model_fh1.h5', custom_objects={'mse_mae': mse_mae})
    gc.collect()
    trainPredict = wind_model.predict(trvaX, batch_size=b_size)
    testPredict = wind_model.predict(teX, batch_size=b_size)

In [14]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [15]:
travaY = trvaY.reshape(-1)
trPredict = trainPredict.reshape(-1)
testY = teY.reshape(-1)
tePredict = testPredict.reshape(-1)

In [16]:
npMSE(trPredict, travaY), npMAE(trPredict, travaY), npMAEMS(trPredict, travaY), npMSE(tePredict, testY), npMAE(tePredict, testY), npMAEMS(tePredict, testY)

(0.0014765326052411203,
 0.02877410665093119,
 0.7134286745694425,
 0.0014714612314261795,
 0.029122771839655996,
 0.5567146628634935)

In [17]:
trPredict = trPredict[(travaY>0.8)]
travaY = travaY[(travaY>0.8)]
tePredict = tePredict[(testY>0.8)]
testY = testY[(testY>0.8)]
npMSE(trPredict, travaY), npMAE(trPredict, travaY), npMAEMS(trPredict, travaY), npMSE(tePredict, testY), npMAE(tePredict, testY), npMAEMS(tePredict, testY)

(0.0009010282421876197,
 0.02286726532605999,
 1.5607565372719516,
 0.0011145096324988134,
 0.023164220776920638,
 1.441543545781946)

In [18]:
errors = trainPredict - trvaY
prediction = trainPredict

In [19]:
norm_df2 = pd.DataFrame(norm_df).iloc[timesteps:trvaX.shape[0]+timesteps, :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)
norm_df2.shape

(31532, 1)

## Load and Test Pre-trained FFEL Model

In [20]:
Prindex = ['Prediction1']
Erindex = ['Error1']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)
norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2

,Normalized Wind,Prediction1,Error1
0,0.726913,0.705698,-0.008974
1,0.714673,0.735707,-0.011706
2,0.747412,0.767117,0.028302
3,0.738814,0.781181,0.073649
4,0.707532,0.730203,0.049193
...,...,...,...
31527,0.272763,0.332515,0.009341
31528,0.323174,0.378504,-0.003796
31529,0.382300,0.430836,-0.015123
31530,0.445958,0.532108,0.017540


In [21]:
norm_df2 = np.array(norm_df2)
Xe, Ye = create_dataset(norm_df2, timesteps, 1, 0)
Xe.shape

(31506, 24, 3)

In [22]:
trvaXe = Xe
trvaYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [23]:
with tf.device('/gpu:0'):
    trainerPredict = winder_model.predict(trvaXe, batch_size=b_size)

In [24]:
addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
#addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [25]:
errors = testPredict - teY
prediction = testPredict

norm_df2 = pd.DataFrame(norm_df).iloc[trvaX.shape[0]+timesteps:trvaX.shape[0]+timesteps+teY.shape[0], :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)

Prindex = ['Prediction1']
Erindex = ['Error1']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)

norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, output_timesteps,0)
teXe = Xe
teYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [26]:
with tf.device('/gpu:0'):
    #trainerPredict = winder_model.predict(trvaXe, batch_size=b_size)
    testerPredict = winder_model.predict(teXe, batch_size=b_size)

In [27]:
#addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [28]:
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-trainPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), )
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-addtrainPredict))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], addtrainPredict), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], addtrainPredict))
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-testPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAEMS == ', np.mean((abs(testPredict[timesteps:-2,:] - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-addtestPredict))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], addtestPredict), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], addtestPredict), ' MAEMS == ', np.mean((abs(addtestPredict - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)

Error Train Score > MSE ==  0.0014734755989471828  MAE ==  0.02875455660224634  MAPE ==  inf
Error Train Score > MSE ==  0.0008121420726668971  MAE ==  0.021362751220787  MAPE ==  inf
Error Test Score > MSE ==  0.0014524701411472067  MAE ==  0.02898474054227111  MAPE ==  11.96688290143424  MAEMS ==  0.11373773684502232
Error Test Score > MSE ==  0.0007381920602187625  MAE ==  0.020467884144792597  MAPE ==  8.26832110476965  MAEMS ==  0.08135952452115607


In [29]:
result_tr = np.concatenate([addtrainPredict.reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1), trainPredict[timesteps:-2,:].reshape(-1,1)], axis=1)
result_te = np.concatenate([addtestPredict.reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1), testPredict[timesteps:-2,:].reshape(-1,1)], axis=1)

result_tr_df = pd.DataFrame(result_tr, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])
result_te_df = pd.DataFrame(result_te, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])

In [30]:
teY[timesteps:-2,:].reshape(-1,1).shape, addtestPredict.reshape(-1,1).shape

((3478, 1), (3478, 1))

In [31]:
result_te_df

,Predicted Value,Real Value,Forecasting Error,Initial Predicted Value
0,0.646594,0.670123,0.023529,0.609773
1,0.665638,0.750445,0.084807,0.606833
2,0.793259,0.797475,0.004215,0.714690
3,0.725028,0.759475,0.034447,0.720495
4,0.675979,0.705090,0.029111,0.670945
...,...,...,...,...
3473,0.278960,0.293595,0.014636,0.287992
3474,0.310148,0.311174,0.001026,0.312126
3475,0.322091,0.341026,0.018935,0.331235
3476,0.349877,0.370742,0.020865,0.335922


In [32]:
error_tr_FFEL = trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1)
error_tr_basic = trvaY[timesteps:-2,:].reshape(-1,1)-trainPredict[timesteps:-2,:].reshape(-1,1)
error_te_FFEL = teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1)
error_te_basic = teY[timesteps:-2,:].reshape(-1,1)-testPredict[timesteps:-2,:].reshape(-1,1)

In [33]:
np.mean(error_tr_FFEL), np.var(error_tr_FFEL), np.mean(error_tr_basic), np.var(error_tr_basic), np.mean(error_te_FFEL),np.var(error_te_FFEL), np.mean(error_te_basic), np.var(error_te_basic)

(0.0006882603798271153,
 0.0008116683703164573,
 -0.0011741212317119846,
 0.0014720970382804258,
 0.001012927875586502,
 0.0007371660373376223,
 0.0014848148186632612,
 0.001450265466101485)

In [34]:
result_tr_df.to_csv('training_result_df_fh1.csv')
result_te_df.to_csv('test_result_df_fh1.csv')

# Ramp Rate Forecasting

In [35]:
norm_df = np.array(wind_df.iloc[:,1]).reshape(-1,1)
timesteps = 24
output_timesteps = 1
leadtime = 1
num_features = 1
X, Y = create_dataset(norm_df, timesteps, output_timesteps, leadtime)
b_size = timesteps

In [36]:
X.shape, Y.shape

((35036, 24, 1), (35036, 1, 1))

In [37]:
dec_num = timesteps
high_ind = 12

addX = np.zeros((X.shape[0], dec_num, high_ind+1))

for k in range(X.shape[0]//dec_num):
    rec_x, low_sig = svdadd(X, k*dec_num, (k+1)*dec_num, b_size)
    addX[k*dec_num:k*dec_num+dec_num,:, :-1] = rec_x
    addX[k*dec_num:k*dec_num+dec_num,:, -1] = low_sig
    
rec_x, low_sig = svdadd(X, X.shape[0]-dec_num, X.shape[0], b_size)
addX[X.shape[0]-dec_num:X.shape[0], :, :-1] = rec_x
addX[X.shape[0]-dec_num:X.shape[0],:, -1] = low_sig

addX_det = addX[:, :, :-1]
addX_det = np.sum(addX_det, axis=2)
X = np.concatenate([X, addX], axis=2)
num_features = X.shape[2]

## Load and Test Pre-trained Basic Model

In [38]:
trvaX, teX, trvaY, teY = train_test_split(X, Y, test_size=0.1, shuffle=False)
trvaY = trvaY.reshape(-1,output_timesteps)
teY = teY.reshape(-1,output_timesteps)
trvaX.shape, trvaY.shape, teX.shape, teY.shape

((31532, 24, 14), (31532, 1), (3504, 24, 14), (3504, 1))

In [39]:
gc.collect()

4178

In [40]:
with tf.device('/gpu:0'):
    b_size = output_timesteps*100
    ramp_model = keras.models.load_model('Basic Ramp Model Final_fh1.h5', custom_objects={'MAEMD_1h': MAEMD_1h})
    ramper_model = keras.models.load_model('Error Learning Ramp Model_fh1.h5', custom_objects={'mse_mae': mse_mae})
    gc.collect()
    trainPredict = ramp_model.predict(trvaX, batch_size=b_size)
    testPredict = ramp_model.predict(teX, batch_size=b_size)

In [41]:
def npMAEMS(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true))*100
def npMAEMD(y_true, y_pred):
    return np.mean((abs(y_pred - y_true))*np.square(y_true-np.mean(y_true)))*100
def npMSE(y_true, y_pred):
    return np.mean(np.square(-y_true+y_pred))
def npMAE(y_true, y_pred):
    return np.mean(abs(-y_true+y_pred))

In [42]:
errors = trainPredict - trvaY
prediction = trainPredict

In [43]:
norm_df2 = pd.DataFrame(norm_df).iloc[timesteps:trvaX.shape[0]+timesteps, :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)
norm_df2.shape

(31532, 1)

## Load and Test Pre-trained FFEL Model

In [44]:
Prindex = ['Prediction1']
Erindex = ['Error1']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)
norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2

,Normalized Wind,Prediction1,Error1
0,0.553489,0.514616,0.029950
1,0.484666,0.514230,-0.073559
2,0.587790,0.538362,0.045344
3,0.493018,0.535952,0.094940
4,0.441011,0.507150,0.055225
...,...,...,...
31527,0.557089,0.651483,0.023180
31528,0.628303,0.682623,0.034338
31529,0.648285,0.762604,0.103929
31530,0.658675,0.697734,0.027710


In [45]:
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, output_timesteps,0)
Xe.shape

(31506, 24, 3)

In [46]:
trvaXe = Xe
trvaYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [47]:
with tf.device('/gpu:0'):
    trainerPredict = ramper_model.predict(trvaXe, batch_size=b_size)

In [48]:
addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
#addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [49]:
errors = testPredict - teY
prediction = testPredict

norm_df2 = pd.DataFrame(norm_df).iloc[trvaX.shape[0]+timesteps:trvaX.shape[0]+timesteps+teY.shape[0], :]
norm_df2.columns = ['Normalized Wind']
norm_df2 = norm_df2.reset_index(drop=True)

Prindex = ['Prediction1']
Erindex = ['Error1']

pr_df = pd.DataFrame(prediction, columns=Prindex)
er_df = pd.DataFrame(errors, columns=Erindex)

norm_df2 = pd.concat([norm_df2, pr_df, er_df],axis=1)
norm_df2 = np.array(norm_df2)
output_timesteps = 1
Xe, Ye = create_dataset(norm_df2, timesteps, output_timesteps,0)
teXe = Xe
teYe = Ye[:,:,-output_timesteps:].reshape(-1,output_timesteps)

In [50]:
with tf.device('/gpu:0'):
    #trainerPredict = winder_model.predict(trvaXe, batch_size=b_size)
    testerPredict = ramper_model.predict(teXe, batch_size=b_size)

In [51]:
#addtrainPredict = trainPredict[timesteps:-2,:]-trainerPredict
addtestPredict = testPredict[timesteps:-2,:]-testerPredict

In [52]:
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-trainPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], trainPredict[timesteps:-2,:]), )
print('Error Train Score > MSE == ', (np.mean(np.square(trvaY[timesteps:-2,:]-addtrainPredict))), ' MAE == ', mean_absolute_error(trvaY[timesteps:-2,:], addtrainPredict), ' MAPE == ', mean_absolute_percentage_error(trvaY[timesteps:-2,:], addtrainPredict))
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-testPredict[timesteps:-2,:]))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], testPredict[timesteps:-2,:]), ' MAEMS == ', np.mean((abs(testPredict[timesteps:-2,:] - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)
print('Error Test Score > MSE == ', (np.mean(np.square(teY[timesteps:-2,:]-addtestPredict))), ' MAE == ', mean_absolute_error(teY[timesteps:-2,:], addtestPredict), ' MAPE == ', mean_absolute_percentage_error(teY[timesteps:-2,:], addtestPredict), ' MAEMS == ', np.mean((abs(addtestPredict - teY[timesteps:-2,:]))*np.square(teY[timesteps:-2,:]-np.mean(teY[timesteps:-2,:])))*100)

Error Train Score > MSE ==  0.006472728215876023  MAE ==  0.05925505117287631  MAPE ==  inf
Error Train Score > MSE ==  0.003916463884504097  MAE ==  0.045729985024268914  MAPE ==  inf
Error Test Score > MSE ==  0.005630743764962211  MAE ==  0.05713346838100066  MAPE ==  12.796280337116784  MAEMS ==  0.08566324032642772
Error Test Score > MSE ==  0.0031991559723526554  MAE ==  0.04227892611108739  MAPE ==  8.801453825321657  MAEMS ==  0.06390341009314367


In [53]:
result_tr = np.concatenate([addtrainPredict.reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1), trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1), trainPredict[timesteps:-2,:].reshape(-1,1)], axis=1)
result_te = np.concatenate([addtestPredict.reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1), teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1), testPredict[timesteps:-2,:].reshape(-1,1)], axis=1)

result_tr_df = pd.DataFrame(result_tr, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])
result_te_df = pd.DataFrame(result_te, columns=['Predicted Value','Real Value','Forecasting Error', 'Initial Predicted Value'])

In [54]:
error_tr_FFEL = trvaY[timesteps:-2,:].reshape(-1,1)-addtrainPredict.reshape(-1,1)
error_tr_basic = trvaY[timesteps:-2,:].reshape(-1,1)-trainPredict[timesteps:-2,:].reshape(-1,1)
error_te_FFEL = teY[timesteps:-2,:].reshape(-1,1)-addtestPredict.reshape(-1,1)
error_te_basic = teY[timesteps:-2,:].reshape(-1,1)-testPredict[timesteps:-2,:].reshape(-1,1)

In [55]:
np.mean(error_tr_FFEL), np.var(error_tr_FFEL), np.mean(error_tr_basic), np.var(error_tr_basic), np.mean(error_te_FFEL),np.var(error_te_FFEL), np.mean(error_te_basic), np.var(error_te_basic)

(0.0029113482659550976,
 0.003907987935778418,
 -0.04133397366692774,
 0.004764230836777747,
 0.0040189416974701075,
 0.003183004079984991,
 -0.03779749458381089,
 0.004202093168148998)

In [56]:
result_tr_df.to_csv('training_ramp_result_df_fh1.csv')
result_te_df.to_csv('test_ramp_result_df_fh1.csv')